# Section 7: Graph, Topological Sort, Shortest Path

# Graph

Graph consists of a finite set of Vertices (or Nodes) and a set of Edges which connect a pair of nodes.

__Terminologies:__

- __Vertices:__ are the nodes of the graph.
- __Edges:__ are the lines (or paths) that connect the vertices of the graphs.
- __Unweighted Graph:__ are graphs that do not have any weight associated with any edge.
- __Weighted Graph:__ are graphs that have weight associated with their edges.
- __Undirected Graph:__ are graphs that the edges do not have directions associated with them.
- __Directed Graph:__ are graphs that the edges have directions associated with them.
- __Cyclic Graph:__ a graph which has at least one loop.
- __Acyclic Graph:__ a graph which does not have any loop.
- __Tree:__ is a special case of __Directed Acyclic Graph__.


__Representations:__

- __Adjacency Matrix:__ is a square matrix or you can say it is a 2D array. And the elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph.
- __Adjacency List:__ is a collection of unorded list used to represent a graph. Each list describes the set of neighbors of a vertex in the graph.

In [1]:
class Graph:
    def __init__(self, graph={}):
        self.graph = graph
        
    def add_edge(self, vertex, edge):
        if vertex in self.graph:
            self.graph[vertex].append(edge)
        else:
            self.graph[vertex] = [edge]
    
adj_lst = {
    "A":["B", "C"],
    "B":["A", "D", "E"],
    "C":["A", "E"],
    "D":["B", "E", "F"],
    "E":["B", "C", "F"],
    "F":["D", "E"],
    
}

graph = Graph(adj_lst)
print(graph.graph)

{'A': ['B', 'C'], 'B': ['A', 'D', 'E'], 'C': ['A', 'E'], 'D': ['B', 'E', 'F'], 'E': ['B', 'C', 'F'], 'F': ['D', 'E']}


# Breadth First Search

It is an algorithm for traversing graph data structure. It starts at some arbitrary vertex of a graph and explores the neighbor nodes (which are at current level) first, before moving to the next level neighbors.

In [2]:
from collections import deque

def breadth_first_search(graph, start):
    dq = deque()
    visited = set()
    result = []
    
    dq.append(start)
    visited.add(start)
    result.append(start)
    
    while dq:
        curr = dq.popleft()

        for nei in graph[curr]:
            if nei not in visited:
                result.append(nei)
                visited.add(nei)
                dq.append(nei)
    
    return result

print(breadth_first_search(adj_lst, "A"))

['A', 'B', 'C', 'D', 'E', 'F']


# Depth First Search

It is an algorithm for traversing graph data structure. It starts at some arbitrary vertex of a graph and explores as far as possible along each edge before backtracking.

In [3]:
# recursive - TC: (v+e), SC: O(v+e)
def depth_first_search(graph, start):
    def _dfs(node):
        if node in visited:
            return
        
        result.append(node)
        visited.add(node)
        for nei in graph[node]:
            _dfs(nei)
    
    result = []
    visited = set()
    _dfs(start)
    return result


def depth_first_search(graph, node, visited=set(), result=[]):
    if node in visited:
        return
    
    result.append(node)
    visited.add(node)
    
    for nei in graph[node]:
        depth_first_search(graph, nei, visited, result)
        
    return result

print(depth_first_search(adj_lst, "A"))

# interative - TC:, SC:
def depth_first_search(graph, start):
    stack = []
    visited = set()
    result = []
    
    stack.append(start)
    visited.add(start)
    
    while stack:
        curr = stack.pop()
        result.append(curr)
        
        for nei in graph[curr]:
            if nei not in visited:
                stack.append(nei)
                visited.add(nei)
    
    return result

print(depth_first_search(adj_lst, "A"))

['A', 'B', 'D', 'E', 'C', 'F']
['A', 'C', 'E', 'F', 'D', 'B']


# Topological Sort

It sorts given actions in such way that if there is a dependency of one action on another, then the dependent action always comes later than its parent action.

In [4]:
def topological_sort(graph):
    def _dfs(node):
        if node in visited:
            return
        
        for nei in graph[node]:
            _dfs(nei)
                
        visited.add(node)
        result.append(node)
    
    result = []
    visited = set()
    for node in graph:
        _dfs(node)
        
    return result[::-1]

tasks = {
    "A":["C"],
    "B":["C", "D"],
    "C":["E"],
    "D":["F"],
    "E":["F", "H"],
    "F":["G"],
    "G":[],
    "H":[],
}

print(topological_sort(tasks))

['B', 'D', 'A', 'C', 'E', 'H', 'F', 'G']


# Single Source Shortest Path Problem

A single source shortest path problem is about finding a path between a given vertex (called source) to all other vertices in the graph such that the total distance between them (source and destination) is minimum.

__The Problem:__

- Five offices in different cities, the cost between these cities are known. Find the cheapest way from head office to branches in different cities.

__Algorithms:__

- BFS
- Dijkstra's Algorithm
- Bellman Ford

In [5]:
# bfs does not work for weighted graphs
# to find shortest path using bfs, we need to enqueue the path instead of only the node
from collections import deque
def bfs(graph, start, end):
    dq = deque()
    visited = set(start)
    path =[start]

    dq.append([start])

    while dq:
        path = dq.popleft()
        node = path[-1]
        
        if node == end:
            return path
            
        for nei in graph[node]:
            if nei not in visited:
                visited.add(nei)
                new_path = list(path)
                new_path.append(nei)
                dq.append(new_path)
            
                

adj_lst = {
    "A":["B", "C"],
    "B":["A", "D", "E"],
    "C":["A", "E"],
    "D":["B", "E", "F"],
    "E":["B", "C", "F"],
    "F":["D", "E"],    
}

bfs(adj_lst, "A", "F")

['A', 'B', 'D', 'F']

# Dijkstra's Algorithm

It finds the shortest path between some start and end node in positive and negative weighted graphs, if there is no any negative cycle.

In [33]:
from collections import deque

def dijkstra(graph, start):
    dq = deque()
    
    aux = {}
    for key in graph:
        aux[key] = {"cost":float("inf"), "prev": []}
        
    aux[start]["cost"] = 0
    
    dq.append(start)
    
    while dq:
        curr = dq.popleft()
        
        for nei in graph[curr]:
            if aux[curr]["cost"] + graph[curr][nei] < aux[nei]["cost"]:
                aux[nei]["cost"] = aux[curr]["cost"] + graph[curr][nei]
                aux[nei]["prev"] += aux[curr]["prev"]
                aux[nei]["prev"].append(curr)
                dq.append(nei)                    

    return aux
    
    
adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

dijkstra(adj, "A")

{'A': {'cost': 0, 'prev': []},
 'B': {'cost': 2, 'prev': ['A']},
 'C': {'cost': 5, 'prev': ['A']},
 'D': {'cost': 3, 'prev': ['A', 'B']},
 'E': {'cost': 5, 'prev': ['A', 'B']},
 'F': {'cost': 13, 'prev': ['A', 'C']},
 'G': {'cost': 14, 'prev': ['A', 'B', 'E']}}

# All Shortest Paths Algorithm


In [40]:
from collections import deque
def find_all_shortest_path(graph):
    def dijkstra(start):
        distances = {}

        for vertex in graph:
            distances[vertex] = {"cost": float("inf"), "prev": None}

        distances[start]["cost"] = 0
        distances[start]["prev"] = start
        dq = deque()
        dq.append(start)

        while dq:
            curr = dq.popleft()

            for nei in graph[curr]:
                if distances[curr]["cost"] + graph[curr][nei] < distances[nei]["cost"]:
                    distances[nei]["cost"] = distances[curr]["cost"] + graph[curr][nei]
                    distances[nei]["prev"] = curr
                    dq.append(nei)

        return distances


In [41]:
adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

dijkstra(adj, "A")

{'A': {'cost': 0, 'prev': 'A'},
 'B': {'cost': 2, 'prev': 'A'},
 'C': {'cost': 5, 'prev': 'A'},
 'D': {'cost': 3, 'prev': 'B'},
 'E': {'cost': 5, 'prev': 'B'},
 'F': {'cost': 13, 'prev': 'C'},
 'G': {'cost': 14, 'prev': 'E'}}

# Minimum Spanning Tree

A minimum spanning tree (MST) is a subset of edges of connected weighted and undirected graph, which:

- connects all vertices together
- no cycle
- minimum total edge

## Disjoint Set

We use Disjoint Set to find cycles in a weighted undirected graph. This data structure is used in the Kruskal's Algorithm.

In [65]:
class DisjointSet:
    def __init__(self, vertices):
        self.vertices = vertices
        self.parent = {v:v for v in vertices}
        self.rank = dict.fromkeys(vertices, 0)
        
    def find(self, item):
        if self.parent[item] == item:
            return item
        return self.find(self.parent[item])
    
    def union(self, x, y):
        xroot = self.find(x)
        yroot = self.find(y)
        
        if self.rank[xroot] < self.rank[yroot]:
            self.parent[xroot] = yroot
        elif self.rank[xroot] > self.rank[yroot]:
            self.parent[yroot] = xroot
        else:
            self.parent[yroot] = xroot
            self.rank[xroot] += 1
    
ds = DisjointSet(adj)

print(ds.find("A"))

ds.union("A", "B")
ds.union("B", "C")
ds.union("E", "B")

print(ds.parent)
print(ds.rank)
print(ds.find("E"))
        

A
{'A': 'A', 'B': 'A', 'C': 'A', 'D': 'D', 'E': 'A', 'F': 'F', 'G': 'G'}
{'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0}
A


# Kruskal's Algorithm

It is a greedy algorithm to find the mininum spanning tree in a graph in two ways:

- add increasing cost edges at each step
- avoid any cycle at each step

Pratical Problems:

- Landing cables
- TV Network
- Tour Operations
- LAN Networks
- A network of pipes for drinking water or natural gas.
- An electric grid
- Single-link Cluster

In [72]:
def create_edges(graph):
    edges = []
    
    for key in graph:
        for v in graph[key]:
            if (v, key, graph[key][v]) not in edges:
                edges.append((key, v, graph[key][v]))
            
    return edges

def kruskal(edges):
    ds = DisjointSet(adj)
    edges = sorted(edges, key=lambda item: item[2])
    total = 0
    mst = []
    for edge in edges:
        if ds.find(edge[0]) != ds.find(edge[1]):
            total += edge[2]
            mst.append(edge)
            ds.union(edge[0], edge[1])
            
    print(total, mst)
            

adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

edges = create_edges(adj)
kruskal(edges)

26 [('B', 'D', 1), ('A', 'B', 2), ('B', 'E', 3), ('A', 'C', 5), ('F', 'G', 7), ('C', 'F', 8)]


# Prim's Algorithm

It is used to find the minimum spanning tree in a graph. It does not use the Disjoint Set data structure.

Pratical Problems:

- Network for roads and Rail tracks connecting all the cities.
- Irrigation channels and placing microwave towers
- Designing a fiber-optic grid or ICs.
- Traveling Salesman Problem.
- Cluster analysis.
- Pathfinding algorithms used in Al(Artificial Intelligence).

In [90]:
from collections import deque

def prim(graph):
    visited = set()
    aux = {vtx:float("inf") for vtx in graph}
    dq = deque()
    dq.append(list(aux)[0])
    aux[list(aux)[0]] = 0
    
    while dq:
        cur = dq.popleft()

        for nei in graph[cur]:
            if nei not in visited:
                if graph[cur][nei] < aux[nei]:
                    aux[nei] = graph[cur][nei]
                visited.add(cur)
                dq.append(nei)
    
    return sum(aux.values())
    

adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

prim(adj)

26

# Kruskal vs Prim

__Kruskal's Algorithm:__

- Landing cables
- TV Network
- Tour Operations
- LAN Networks
- A network of pipes for drinking water or natural gas.
- An electric grid
- Single-link Cluster

__Prim's Algorithm:__

- Network for roads and Rail tracks connecting all the cities.
- Irrigation channels and placing microwave towers
- Designing a fiber-optic grid or ICs.
- Traveling Salesman Problem.
- Cluster analysis.
- Pathfinding algorithms used in Al(Artificial Intelligence)